# 04 - Results and Visualization

This notebook provides comprehensive visualization and analysis of results from both Genetic Algorithm and MIP approaches.

## Import Libraries

In [ ]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_utils import load_attractions_data
from visualization import (
    plot_fitness_evolution,
    plot_route_on_map,
    plot_tour_statistics,
    plot_distance_matrix,
    compare_algorithms,
    create_summary_report
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Data and Results

In [ ]:
# Load attractions data
attractions = load_attractions_data('../data/sri_lanka_attractions.csv')
distance_matrix = np.load('../data/distance_matrix.npy')

print(f"Loaded {len(attractions)} attractions")

# Load algorithm results
try:
    ga_results = np.load('../data/ga_results.npy', allow_pickle=True).item()
    print("GA results loaded successfully")
except:
    ga_results = None
    print("GA results not found")

try:
    mip_results = np.load('../data/mip_results.npy', allow_pickle=True).item()
    print("MIP results loaded successfully")
except:
    mip_results = None
    print("MIP results not found")

## Overview of Tourist Attractions

In [ ]:
# Display attractions table
print("Sri Lankan Tourist Attractions Dataset")
print("=" * 80)
attractions_display = attractions.copy()
attractions_display.index = range(1, len(attractions) + 1)
attractions_display

In [ ]:
# Summary statistics
print("\nDataset Summary Statistics:")
print("=" * 80)
print(f"Total number of attractions: {len(attractions)}")
print(f"\nScore Statistics:")
print(f"  Mean: {attractions['score'].mean():.2f}")
print(f"  Median: {attractions['score'].median():.2f}")
print(f"  Min: {attractions['score'].min():.2f}")
print(f"  Max: {attractions['score'].max():.2f}")
print(f"\nVisit Duration Statistics (hours):")
print(f"  Mean: {attractions['visit_duration'].mean():.2f}")
print(f"  Median: {attractions['visit_duration'].median():.2f}")
print(f"  Min: {attractions['visit_duration'].min():.2f}")
print(f"  Max: {attractions['visit_duration'].max():.2f}")
print(f"  Total: {attractions['visit_duration'].sum():.2f}")

## Visualize Distance Relationships

In [ ]:
# Plot distance matrix heatmap
fig = plot_distance_matrix(distance_matrix, attractions['name'].tolist())
plt.show()

## Genetic Algorithm Results

In [ ]:
if ga_results:
    print("Genetic Algorithm Performance")
    print("=" * 80)
    print(f"Best Fitness: {ga_results['best_fitness']:.2f}")
    print(f"Computation Time: {ga_results['computation_time']:.2f} seconds")
    print(f"Number of Attractions in Tour: {len(ga_results['valid_tour'])}")
    
    print("\nGA Parameters:")
    for param, value in ga_results['parameters'].items():
        print(f"  {param}: {value}")
    
    # Plot fitness evolution
    print("\nFitness Evolution:")
    fig = plot_fitness_evolution(ga_results['fitness_history'])
    plt.show()
    
    # Plot tour statistics
    print("\nTour Statistics:")
    fig = plot_tour_statistics(attractions, ga_results['valid_tour'], distance_matrix)
    plt.show()
    
    # Print tour details
    print("\nGA Tour Sequence:")
    for i, idx in enumerate(ga_results['valid_tour']):
        print(f"{i+1}. {attractions.iloc[idx]['name']} (Score: {attractions.iloc[idx]['score']}, Duration: {attractions.iloc[idx]['visit_duration']}h)")
else:
    print("GA results not available. Run notebook 02 to generate results.")

## MIP Model Results

In [ ]:
if mip_results:
    print("Mixed Integer Programming Performance")
    print("=" * 80)
    print(f"Total Score: {mip_results['total_score']:.2f}")
    print(f"Computation Time: {mip_results['computation_time']:.2f} seconds")
    print(f"Solution Status: {mip_results['status']}")
    print(f"Number of Attractions in Tour: {len(mip_results['tour_sequence'])}")
    
    print("\nMIP Parameters:")
    for param, value in mip_results['parameters'].items():
        print(f"  {param}: {value}")
    
    # Plot tour statistics
    print("\nTour Statistics:")
    fig = plot_tour_statistics(attractions, mip_results['tour_sequence'], distance_matrix)
    plt.show()
    
    # Print tour details
    print("\nMIP Tour Sequence:")
    for i, idx in enumerate(mip_results['tour_sequence']):
        print(f"{i+1}. {attractions.iloc[idx]['name']} (Score: {attractions.iloc[idx]['score']}, Duration: {attractions.iloc[idx]['visit_duration']}h)")
else:
    print("MIP results not available. Run notebook 03 to generate results.")

## Algorithm Comparison

In [ ]:
if ga_results and mip_results:
    # Prepare comparison data
    comparison_data = {
        'Genetic Algorithm': {
            'score': ga_results['best_fitness'],
            'computation_time': ga_results['computation_time']
        },
        'MIP (PuLP)': {
            'score': mip_results['total_score'],
            'computation_time': mip_results['computation_time']
        }
    }
    
    # Plot comparison
    fig = compare_algorithms(comparison_data)
    plt.show()
    
    # Detailed comparison table
    comparison_df = pd.DataFrame({
        'Algorithm': ['Genetic Algorithm', 'MIP (PuLP)'],
        'Total Score': [ga_results['best_fitness'], mip_results['total_score']],
        'Attractions Visited': [len(ga_results['valid_tour']), len(mip_results['tour_sequence'])],
        'Computation Time (s)': [ga_results['computation_time'], mip_results['computation_time']]
    })
    
    print("\nDetailed Algorithm Comparison:")
    print("=" * 80)
    print(comparison_df.to_string(index=False))
    print("=" * 80)
    
    # Calculate differences
    score_diff = abs(ga_results['best_fitness'] - mip_results['total_score'])
    score_diff_pct = (score_diff / max(ga_results['best_fitness'], mip_results['total_score'])) * 100
    time_ratio = ga_results['computation_time'] / mip_results['computation_time']
    
    print(f"\nKey Insights:")
    print(f"  Score difference: {score_diff:.2f} ({score_diff_pct:.1f}%)")
    print(f"  Time ratio (GA/MIP): {time_ratio:.2f}x")
    
    if ga_results['best_fitness'] > mip_results['total_score']:
        print(f"  GA found a better solution by {score_diff:.2f} points")
    elif mip_results['total_score'] > ga_results['best_fitness']:
        print(f"  MIP found a better solution by {score_diff:.2f} points")
    else:
        print(f"  Both algorithms found solutions with equal scores")
    
elif ga_results:
    print("Only GA results available. Run notebook 03 to generate MIP results for comparison.")
elif mip_results:
    print("Only MIP results available. Run notebook 02 to generate GA results for comparison.")
else:
    print("No results available. Run notebooks 02 and 03 first.")

## Side-by-Side Tour Comparison

In [ ]:
if ga_results and mip_results:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # GA tour
    ga_tour = attractions.iloc[ga_results['valid_tour']]
    axes[0].barh(range(len(ga_tour)), ga_tour['score'].values, color='steelblue')
    axes[0].set_yticks(range(len(ga_tour)))
    axes[0].set_yticklabels([f"{i+1}. {name[:20]}" for i, name in enumerate(ga_tour['name'])])
    axes[0].set_xlabel('Satisfaction Score')
    axes[0].set_title(f'Genetic Algorithm Tour\n(Total Score: {ga_results["best_fitness"]:.2f})')
    axes[0].grid(True, alpha=0.3)
    
    # MIP tour
    mip_tour = attractions.iloc[mip_results['tour_sequence']]
    axes[1].barh(range(len(mip_tour)), mip_tour['score'].values, color='coral')
    axes[1].set_yticks(range(len(mip_tour)))
    axes[1].set_yticklabels([f"{i+1}. {name[:20]}" for i, name in enumerate(mip_tour['name'])])
    axes[1].set_xlabel('Satisfaction Score')
    axes[1].set_title(f'MIP Tour\n(Total Score: {mip_results["total_score"]:.2f})')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Interactive Maps

In [ ]:
print("Interactive map files:")
print("=" * 80)

if ga_results:
    print("✓ GA Tour Map: ../data/ga_tour_map.html")
    print("  Open in browser to view the Genetic Algorithm tour route")

if mip_results:
    print("✓ MIP Tour Map: ../data/mip_tour_map.html")
    print("  Open in browser to view the MIP tour route")

if not ga_results and not mip_results:
    print("No maps available. Run notebooks 02 and 03 to generate tour maps.")

## Performance Analysis

In [ ]:
# Analyze solution quality vs computation time trade-off
if ga_results and mip_results:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    algorithms = ['GA', 'MIP']
    scores = [ga_results['best_fitness'], mip_results['total_score']]
    times = [ga_results['computation_time'], mip_results['computation_time']]
    colors = ['steelblue', 'coral']
    
    # Scatter plot
    for i, algo in enumerate(algorithms):
        ax.scatter(times[i], scores[i], s=300, c=colors[i], alpha=0.6, edgecolors='black', linewidth=2)
        ax.annotate(algo, (times[i], scores[i]), 
                   xytext=(10, 10), textcoords='offset points',
                   fontsize=12, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor=colors[i], alpha=0.3))
    
    ax.set_xlabel('Computation Time (seconds)', fontsize=12)
    ax.set_ylabel('Solution Quality (Total Score)', fontsize=12)
    ax.set_title('Solution Quality vs Computation Time', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Conclusions and Recommendations

In [ ]:
print("=" * 80)
print("CONCLUSIONS AND RECOMMENDATIONS")
print("=" * 80)

if ga_results and mip_results:
    print("\n1. Solution Quality:")
    if abs(ga_results['best_fitness'] - mip_results['total_score']) < 0.5:
        print("   • Both algorithms produced similar quality solutions")
    elif ga_results['best_fitness'] > mip_results['total_score']:
        print("   • GA produced a higher-scoring solution")
    else:
        print("   • MIP produced a higher-scoring solution (likely optimal)")
    
    print("\n2. Computational Efficiency:")
    if ga_results['computation_time'] < mip_results['computation_time']:
        print("   • GA was faster to compute")
    else:
        print("   • MIP was faster to compute")
    
    print("\n3. Recommendations:")
    print("   • For small-scale problems (<20 attractions): Use MIP for optimal solutions")
    print("   • For large-scale problems (>50 attractions): Use GA for good solutions quickly")
    print("   • For real-time applications: Use GA for faster response times")
    print("   • For critical planning: Use both and compare results")
    
    print("\n4. Future Improvements:")
    print("   • Implement hybrid approaches (GA + local search)")
    print("   • Add multi-day tour planning capabilities")
    print("   • Include user preferences and constraints")
    print("   • Implement parallel GA for faster computation")
    print("   • Add real-time traffic and weather considerations")

print("\n" + "=" * 80)

## Export Results Summary

In [ ]:
# Create comprehensive summary
summary = "TOURIST TRIP DESIGN PROBLEM - RESULTS SUMMARY\n"
summary += "=" * 80 + "\n\n"

summary += "Dataset Information:\n"
summary += f"  Total Attractions: {len(attractions)}\n"
summary += f"  Average Score: {attractions['score'].mean():.2f}\n"
summary += f"  Average Visit Duration: {attractions['visit_duration'].mean():.2f} hours\n\n"

if ga_results:
    summary += "Genetic Algorithm Results:\n"
    summary += f"  Best Fitness: {ga_results['best_fitness']:.2f}\n"
    summary += f"  Attractions Visited: {len(ga_results['valid_tour'])}\n"
    summary += f"  Computation Time: {ga_results['computation_time']:.2f} seconds\n\n"

if mip_results:
    summary += "MIP Model Results:\n"
    summary += f"  Total Score: {mip_results['total_score']:.2f}\n"
    summary += f"  Attractions Visited: {len(mip_results['tour_sequence'])}\n"
    summary += f"  Computation Time: {mip_results['computation_time']:.2f} seconds\n\n"

summary += "=" * 80 + "\n"

print(summary)

# Save to file
with open('../data/results_summary.txt', 'w') as f:
    f.write(summary)

print("Summary saved to ../data/results_summary.txt")

## End of Analysis

This concludes the comprehensive analysis of the Tourist Trip Design Problem for Sri Lankan attractions using both Genetic Algorithm and Mixed Integer Programming approaches.